# U-NET modified for density kernel estimation


U-NET is a popular end-to-end segmentation net used in biomedical imaging [1]. Following a suggestion from a friend(Weidi Xie) I wanted to see if this network structure could be used to perform density estimation. I took the U-NET implementation from Weidi. With some small changes and some basic pre-processing I have been able to achieve close to state-of-the-art performance.

#### Background:

By using machine learning it is possible to learn a mapping from intensity to a density kernel representation of cells and objects. Through integration of the densities one can then predict the number of cells of objects in a scene without having to explicity segment the objects of interest. [2] To do this accurately, for typical images, it is necessary to characterise the image pixels by calculating long feature descriptors which can abstract image information in different ways and at different scales. Neural networks can handle large training sets and learn features which are tuned for specific applications. Many different networks can be applied to this problem however the current state-of-the-art for density kernel estimation in 2-D is a Fully Convolutional Regression Neural Network [3] which comprises of convolutional layers along with the normal activation and pooling layers, but then has upsampling layers which allow the output of the network to expand to be in parity with the input dimension. U-NET is a fully convolutional network also but it uses concatenations of layers and a highly symmetrical network design. I was interested to see if this could be directly applied to the density estimation and how well it would peform.

#### Changes:

-- The loss function used is 'mse' not 'cross-entropy'.

-- As I am not performing segmentation so don't use softmax and use relu rather than sigmoid in the last activation layer.

-- As mentioned in [2] it is important to scale the density kernels from 0.0-1.0 to something higher ( in this case 0.0-255.0) otherwise the network struggles to learn the very small target values produced by Gaussian convolution. These values are down-scaled subsequently to valid densities (0.0-1.0).

#### Datasets:

I used one some of a dataset which I have recieved from a collaborator (Caroline Scott). These are cells imaged using bright-field microscopy. We are interested in how many cells are present in each image.


#### Bibliography:

[1] Ronneberger, Olaf, Philipp Fischer, and Thomas Brox. "U-net: Convolutional networks for biomedical image segmentation." International Conference on Medical Image Computing and Computer-Assisted Intervention. Springer International Publishing, 2015.

[2] Lempitsky, Victor, and Andrew Zisserman. "Learning to count objects in images." Advances in Neural Information Processing Systems. 2010.

[3] Xie, Weidi, J. Alison Noble, and Andrew Zisserman. "Microscopy Cell Counting with Fully Convolutional Regression Networks."

In [9]:
%pylab inline

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #Ensures I only use one of my available GPUs.
from random import Random
import cPickle as pickle


import tensorflow as tf
import keras
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.backend.tensorflow_backend import set_session
from keras.models import Sequential, Model, load_model


config = tf.ConfigProto()
#Uses 80% of the memory of one GPU, rather than all the memory of all the GPUs
config.gpu_options.per_process_gpu_memory_fraction = 0.8 
set_session(tf.Session(config=config))


#Some other specialist functions which could easily be substituted.
from PIL import Image #for image import and output
from scipy import ndimage #for Gaussian filtering
import cv2 #for padding function of image border.
from sklearn.model_selection import train_test_split

from model import buildModel #additional functions
from generator import ImageDataGenerator, split_the_images #additional functions



class TestCallback(Callback):
    """This function reports back the progress of the learning."""
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
    def on_train_begin(self, logs={}):
        self.losses = []
        
def step_decay(epoch):
    """This sets up the various decays of the learning rate."""
    step = 16
    num =  epoch // step
   
    if num >2:
         lrate = 1e-6
    else:
        if num % 3 == 0:
            lrate = 1e-3
        elif num % 3 == 1:
            lrate = 1e-4
        elif num % 3:
            lrate = 1e-5
    print('Learning rate for epoch {} is {}.'.format(epoch+1, lrate))
    return np.float(lrate)

def dice_coef(y_true, y_pred):
    """This is the default U-NET loss function which I do not use."""
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    smooth =1.0
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def get_unet(img_rows,img_cols):
    return buildModel(input_dim = (img_rows,img_cols,1))

def return_shuffled_im(train_size, test_size, input_seq):
    """returns shuffled indices for test and training data."""
    input_seq = input_seq.astype(np.int32)
    image_pool_size = np.array(input_seq).shape
    rand_seq = np.random.choice(input_seq,input_seq.shape[0],replace=True)

    assert (train_size + test_size) <= image_pool_size,\
    'your combined test and train images exceeds the number of images in your input sequence.'

    shuf_train_images = rand_seq[:train_size]
    shuf_test_images = rand_seq[train_size:train_size+test_size]


    return shuf_train_images, shuf_test_images



Populating the interactive namespace from numpy and matplotlib


# Code for training algorithm

In [ ]:
#generate how many models.
models_to_gen = 5

for bt in range(0,models_to_gen):
    
    file_path = 'dataset02/' #Dataset to use.
    data_store = {}
    data_store['input'] = []
    data_store['gt'] = []
    data_store['dense'] = []
    
    #Parameters of fit.
    in_hei = 96 #Size of each image patch
    in_wid = 96 #Size of each image patch
    mag = 16 #The padding margin to use
    train_size = 62 #The number of training images.
    test_size = 18 #The number of test images.
    learning_rate ="1e-5 see learning rate decay"
    batch_size = 16 #For stochastic gradient descent, the number of images in each batch.
    nb_epoch = 100 #The number of epochs
    samples_per_epoch = 150 #For stochastic gradient descent, the number of batches per epoch.
    image_pool_size = train_size+test_size
    input_seq = np.arange(0,image_pool_size)
    sigma = 2.0 #Size of kernel representing the ground-truth density.
    save_best_only = False
    loss = 'mse' #I use mean square error in this case.
   
    decay = 0.0
    
    
    #Experiment number should match filename
    filepath="saved_models/keras_tf_exp_sdataset02s2_model_"+str(bt)
    
    for i in range(0,image_pool_size):
        n = str(i+1).zfill(3)
        
        #Open intensity image.
        img = Image.open(file_path+n+'cells.png').getdata()
        wid, hei = img.size
        temp = np.array(img).reshape((hei,wid,3))[:,:,2].astype(np.float32)
        
        data_store['input'].append(temp)
        
        #Open ground-truth image.
        img =  Image.open(file_path+n+'dots.png').getdata()
        data_store['gt'].append(np.array(img).reshape((hei,wid))[:,:].astype(np.float64))
        
        #Filter ground-truth image to produce density kernel representation
        data_store['dense'].append(ndimage.filters.gaussian_filter(data_store['gt'][i],sigma,mode='constant'))


    train = []
    gtdata = []

    shuf_train_images, shuf_test_images = return_shuffled_im(train_size,test_size,input_seq)
    
    X_trainf = []
    Y_trainf = []
    X_testf = []
    Y_testf = []

    for i in shuf_train_images:
        X_trainf.append(data_store['input'][i])
        Y_trainf.append(data_store['dense'][i])
    for i in shuf_test_images:
        X_testf.append(data_store['input'][i])
        Y_testf.append(data_store['dense'][i])

    #X_trainf, X_testf, Y_trainf, Y_testf = train_test_split(data_store['input'],  data_store['dense'], train_size = train_size,test_size=18)
    train_cut, train_gtdata_cut, images_per_image = split_the_images(X_trainf, Y_trainf,in_hei,in_wid,mag)
    test_cut, test_gtdata_cut, images_per_image = split_the_images(X_testf, Y_testf,in_hei,in_wid,mag)
    

    X_train = np.array(train_cut)
    Y_train = np.array(train_gtdata_cut)  
    X_test = np.array(test_cut)
    Y_test = np.array(test_gtdata_cut)


    #Stores history of paramaters.
    hist = keras.callbacks.History()


    # combine generators into one which yields image and masks
    #train_generator = zip(image_generator, mask_generator)
    height = X_train.shape[2]
    width = X_train.shape[3]
    model = get_unet(height, width)

    X_train = np.swapaxes(X_train,1,3)
    X_train = np.swapaxes(X_train,2,1)
    Y_train = np.swapaxes(Y_train,1,3)
    Y_train = np.swapaxes(Y_train,2,1)
    X_test = np.swapaxes(X_test,1,3)
    X_test = np.swapaxes(X_test,2,1)
    Y_test = np.swapaxes(Y_test,1,3)
    Y_test = np.swapaxes(Y_test,2,1)
   
    
    print 'fitting model',X_train.shape,np.max(X_train)
    checkpoint = ModelCheckpoint(filepath+".hdf5",  monitor='loss', verbose=1, save_best_only=save_best_only, mode='min')


    
    datagen = ImageDataGenerator(
        featurewise_center = False,  # set input mean to 0 over the dataset
        samplewise_center = False,  # set each sample mean to 0
        featurewise_std_normalization = False,  # divide inputs by std of the dataset
        samplewise_std_normalization = False,  # divide each input by its std
        zca_whitening = False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range = 0.3,  # randomly shift images horizontally (fraction of total width)
        height_shift_range = 0.3,  # randomly shift images vertically (fraction of total height)
        zoom_range = 0.3,
        shear_range = 0.,
        horizontal_flip = True,  # randomly flip images
        vertical_flip = True,
        fill_mode = 'constant',
        dim_ordering = 'tf')  # randomly flip images


    change_lr = LearningRateScheduler(step_decay)

    callbacks_list = [checkpoint, change_lr]



    hist = model.fit_generator(datagen.flow(X_train,Y_train,batch_size = batch_size), samples_per_epoch = samples_per_epoch, nb_epoch=nb_epoch, callbacks=callbacks_list, verbose=True, validation_data=(X_test, Y_test))
    
    #Store the paramters of the fit, along with the model and data preprocessing.
    hist.history['parameters'] = {}
    hist.history['parameters']['batch_size'] = batch_size
    hist.history['parameters']['nb_epoch'] = nb_epoch
    hist.history['parameters']['in_hei'] = in_hei
    hist.history['parameters']['in_wid'] = in_wid
    hist.history['parameters']['mag'] = mag
    hist.history['parameters']['sigma'] = sigma
    hist.history['parameters']['train_size'] = train_size
    hist.history['parameters']['learning_rate'] = learning_rate
    hist.history['parameters']['shuf_train_images'] = shuf_train_images
    hist.history['parameters']['shuf_test_images'] = shuf_test_images
    hist.history['parameters']['loss'] = loss
    hist.history['parameters']['save_best_only'] = save_best_only
    pickle.dump(hist.history,open(filepath+'.his',"wb"))



fitting model (1488, 128, 128, 1) 20.385
Learning rate for epoch 1 is 0.001.
Epoch 1/100
9/9 [==============================] - 502s - loss: 1.0884 - val_loss: 0.9660
Learning rate for epoch 2 is 0.001.
Epoch 2/100
9/9 [==============================] - 161s - loss: 0.7539 - val_loss: 0.9707
Learning rate for epoch 3 is 0.001.
Epoch 3/100
9/9 [==============================] - 165s - loss: 0.6139 - val_loss: 0.9693
Learning rate for epoch 4 is 0.001.
Epoch 4/100
1/9 [==>...........................] - ETA: 69s - loss: 0.4484

# Code for making predictions

In [14]:
##Load in the paramters.

model_path ='saved_models'
experiment = 'keras_tf_exp_sdataset02s2_model_'

imported_pickle = pickle.load(open(model_path+'/'+experiment+'0.his',"rb"))

parameters = imported_pickle['parameters']
file_path = 'dataset02/'
data_store = {}
data_store['input'] = []
data_store['gt'] = []
data_store['dense'] = []
train = []
gtdata = []
in_hei = parameters['in_hei']
in_wid = parameters['in_wid']
mag = parameters['mag']
learning_rate = parameters['learning_rate']
#get_unet = parameters['unet']
loss = parameters['loss'] 
save_best_only = parameters['save_best_only']



#optimizer_fn = parameters['optimizer']
#model_fn = parameters['model']
loss = 'mse'
sigma = parameters['sigma']
#metrics = [accuracy_custom,accuracy_custom2]
num_of_train = 19
test_size = 18

   
#optimizer = optimizer_fn(lr=learning_rate,momentum=0.9,decay=0.1)

##Load the images for testing.
for i in range(0,num_of_train):
    n = str(i+80).zfill(3)
    
    #Open intensity image.
    img = Image.open(file_path+n+'cells.png').getdata()
    wid,hei = img.size
    temp = np.array(img).reshape((hei,wid,3))[:,:,2].astype(np.float32)
    
    data_store['input'].append(temp)
    
    #Open ground-truth image.
    img =  Image.open(file_path+n+'dots.png').getdata()
    data_store['gt'].append(np.array(img).reshape((hei,wid))[:,:].astype(np.float64))
    
    #Filter ground-truth image to produce density kernel representation
    data_store['dense'].append(ndimage.filters.gaussian_filter(data_store['gt'][i],sigma,mode='constant'))




X_trainf, X_testf, Y_trainf, Y_testf = train_test_split(data_store['input'],  data_store['dense'], train_size =1,test_size=test_size)

test_cut, test_gtdata_cut,images_per_image = split_the_images(X_testf, Y_testf,in_hei,in_wid,mag)

X_test = np.array(test_cut)
Y_test = np.array(test_gtdata_cut)  



X_test = np.swapaxes(X_test,1,3)
X_test = np.swapaxes(X_test,2,1)

for vt in range(0,5):
    filename=model_path+"/"+experiment+""+str(vt)+".hdf5"
    # load json and create model


        


    # load weights into new model
    loaded_model = load_model(filename)
    print("Loaded model from disk")
    imported_pickle = pickle.load(open(model_path+'/'+experiment+""+str(vt)+'.his',"rb"))

    f = open(model_path+'/'+experiment+str(vt)+'out.txt', 'w+')

    loss_print = imported_pickle['loss']
    val_loss_print = imported_pickle['val_loss']
    f.write('loss_pt\tval_pt\n')
    for loss_pt, val_pt in zip(loss_print, val_loss_print):
            f.write(str(loss_pt)+'\t'+str(val_pt)+'\n')
    f.close()
    #print 'final train loss', imported_pickle['loss'][-1]
    #print 'final train accuracy', imported_pickle['accuracy_custom'][-1]
     
    # evaluate loaded model on test data
    #loaded_model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    imgs_mask_test = loaded_model.predict(X_test, verbose=1)
    #Switches format for tensor flow.
    imgs_mask_test = np.swapaxes(imgs_mask_test,2,1)
    imgs_mask_test = np.swapaxes(imgs_mask_test,1,3)
    data_store['output'] = []
    c =0

    null,f_hei, f_wid = imgs_mask_test[0].shape
    
    #reconstruct the images.

    while c < (imgs_mask_test.__len__()-images_per_image)+1:
        out = np.zeros((hei,wid))
        rows = 0
        for rst in range(0,hei,in_hei):
            rows +=1
            cols = 0
            d = np.floor(float(c)/float(images_per_image))
            for cst in range(0,wid,in_wid):
                cols +=1
                img = imgs_mask_test[c]
                c +=1
                top = bottom = left = right = 16
                ren = rst + in_hei
                cen = cst + in_wid
                if ren> hei:

                    top = top + (ren-hei)
                    ren = hei
                if cen> wid:


                    right = right + (cen-wid)
                    cen = wid

                out[rst:ren,cst:cen] = img[0,bottom:-top,left:-right]
        
        data_store['output'].append(out)
        
    


    #Output of performance metrics


    pef = []
    trk = []
    abs_err = []
    inpu_arr = []
    pred_arr = []
    perc_arr = []

    print data_store['output'].__len__()
    print test_size
    #figsize(12,12)
    for idx in range(0,test_size):
        #figure()
        inpu_img = Y_testf[idx]
        pred_img = data_store['output'][idx]
        n = str(idx)
        b = n.zfill(3)
        pickle.dump( inpu_img, open( "out_imgs/inpu_img"+b+".p", "wb" ) )
        pickle.dump( pred_img, open( "out_imgs/pred_img"+b+".p", "wb" ) )
        #subplot(1,2,1)
        #imshow(inpu_img)
        #subplot(1,2,2)
        #imshow(pred_img)
        inpu_sum = np.sum(inpu_img)/255.0
        pred_sum = np.sum(pred_img)/255.0
        #print 'img',idx
        #print inpu_sum
        #print pred_sum
        #print abs(inpu_sum-pred_sum)
        inpu_arr.append(inpu_sum)
        pred_arr.append(pred_sum)
        abs_err.append(abs(inpu_sum-pred_sum))
        perc_arr.append((1-(abs(inpu_sum-pred_sum)/pred_sum))*100)
        #print np.round((1-(abs(inpu_sum-pred_sum)/pred_sum))*100,0),'%'
    print np.average(abs_err)
    print np.average(inpu_arr)
    print 'average pred',np.average(pred_arr)
    print 'average perc',np.average(perc_arr)
    imported_pickle['pred_arr'] = pred_arr
    imported_pickle['perc_arr'] = perc_arr
    imported_pickle['abs_err'] = pred_arr
    imported_pickle['inpu_arr'] = perc_arr
    #imported_pickle['img_inpu_arr'] = Y_testf
    #imported_pickle['img_pred_arr'] = data_store['output']

    pickle.dump(imported_pickle,open(model_path+'/'+experiment+""+str(vt)+'.his',"w"))


{'inpu_arr': [96.548259897475603, 94.437600363062685, 78.179729460864777, 99.268087152154877, 80.368131661639282, 96.992953637689368, 93.850181568048768, 93.812998783661172, 98.916281612723438, 93.795125003377549, 93.875230967193716, 80.947793589766263, 90.262105397745515, 76.207503915211674, 95.205622060159428, 82.523915063049927, 87.119067184423869, 89.999418922519155], 'loss': [0.61385537683963776, 0.44317046403884885, 0.46911012977361677, 0.42480987161397932, 0.39507896155118943, 0.34107576310634613, 0.35019675642251968, 0.2892808221280575, 0.22749567180871963, 0.35392936468124392, 0.31145465672016143, 0.36326146870851517, 0.36998029649257658, 0.33873944059014321, 0.33421666622161866, 0.31594151556491851, 0.31144656166434287, 0.28976163566112517, 0.26960971504449843, 0.27353150099515916, 0.33533056676387785, 0.32221672981977462, 0.3706328496336937, 0.30823867917060854, 0.35346551835536955, 0.31134958118200301, 0.28328273743391036, 0.21871690228581428, 0.27633555680513383, 0.3234039

ValueError: Improper config format: {u'l2': 9.999999747378752e-05, u'name': u'L1L2Regularizer', u'l1': 0.0}